<a href="https://colab.research.google.com/github/HYUNSOOLEE-6839/colab-Deep-Learning/blob/main/41_%EB%8B%A4%EA%B0%81%ED%98%95%EB%B6%84%EB%A5%98_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 원, 삼각형, 사각형 구분하기

In [3]:
!unzip -qq handwriting_shape.zip -d handwriting_shape

In [4]:
!unzip -qq hard_handwriting_shape.zip -d hard_handwriting_shape

### 1. CNN 모델

In [5]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [6]:
# seed 값 설정
seed = 2021
np.random.seed(seed)
tf.random.set_seed(seed)

### 데이터셋 생성하기

In [21]:
# Numpy Array로 만드는 과정
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'handwriting_shape/train',
    target_size=(24,24),
    batch_size=3,
    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    'handwriting_shape/test',
    target_size=(24,24),
    batch_size=3,
    class_mode='categorical')

Found 45 images belonging to 3 classes.
Found 15 images belonging to 3 classes.


In [8]:
!ls handwriting_shape/train

circle	rectangle  triangle


In [9]:
!ls handwriting_shape/test

circle	rectangle  triangle


In [10]:
train_generator.labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2], dtype=int32)

In [19]:
train_generator.filenames[0]

'circle/circle001.png'

In [20]:
test_generator.filenames[0]

'circle/circle016.png'

### 모델 정의/설정/학습

In [22]:
model = Sequential([
    Conv2D(32, (3,3), input_shape=(24, 24, 3), activation='relu'),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(3, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 22, 22, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 10, 10, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6400)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               819328    
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 387       
Total params: 839,107
Trainable params: 839,107
Non-trainable params: 0
__________________________________________________

In [23]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### 모델 학습시키기
- 첫 번째 인자 : 훈련데이터셋을 제공할 제네레이터를 지정.
- steps_per_epoch : 한 epoch에 사용한 스텝 수를 지정. 총 45개의 훈련 샘플이 있고 배치사이즈가 3이므로 15스텝으로 지정
- epochs : 전체 훈련 데이터 셋에 대해 학습 반복 횟수를 지정. 50번 반복학습
- validation_data : 검증데이터셋을 제공할 제네레이터를 지정. 본 예제에서는 앞서 생성한 validation_generator로 지정.
- validation_steps : 한 epoch 종료 시 마다 검증할 때 사용되는 검증 스텝 수를 지정. 총 15개의 검증 샘플이 있고 배치사이즈가 3이므로 5스텝으로 지정.

In [24]:
history=model.fit_generator(
    train_generator,
    steps_per_epoch=15,
    epochs=50,
    validation_data=test_generator,
    validation_steps=5)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
15/15 [==============================] - 1s 34ms/step - loss: 1.2396 - accuracy: 0.4481 - val_loss: 0.7822 - val_accuracy: 0.6667
Epoch 2/50
15/15 [==============================] - 0s 20ms/step - loss: 0.5481 - accuracy: 0.8984 - val_loss: 0.1336 - val_accuracy: 1.0000
Epoch 3/50
15/15 [==============================] - 0s 22ms/step - loss: 0.0532 - accuracy: 1.0000 - val_loss: 0.0292 - val_accuracy: 1.0000
Epoch 4/50
15/15 [==============================] - 0s 21ms/step - loss: 0.0027 - accuracy: 1.0000 - val_loss: 0.0145 - val_accuracy: 1.0000
Epoch 5/50
15/15 [==============================] - 0s 20ms/step - loss: 7.4627e-04 - accuracy: 1.0000 - val_loss: 0.0134 - val_accuracy: 1.0000
Epoch 6/50
15/15 [==============================] - 0s 20ms/step - loss: 4.7590e-04 - accuracy: 1.0000 - val_loss: 0.0081 - val_accuracy: 1.0000
Epoch 7/50
15/15 [==============================] - 0s 20ms/step - loss: 1.3734e-04 - accuracy: 1.0000 - val_loss: 0.0069 - val_accuracy: 1.0000
E

### 모델 평가하기

In [26]:
print("--Evaluate--")
scores = model.evaluate_generator(test_generator, steps=5)
print("%s: %.2f%%"%(model.metrics_names[1], scores[1]*100));

--Evaluate--
accuracy: 100.00%


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '
